实验八 Linux环境网络管理
=======

# 1. 实验类型

验证型，4学时，必做实验


# 2.  实验目的

了解Linux下的网络功能，验证一个网络服务的配置，理解Linux强大的网络功能。

# 3. 题目描述

本次实验介绍了Linux环境的网络功能，验证了如何安装一个网络服务。

# 4. 实验要求

*基本层次：要求学生能实践指导书的例子，体会网络服务的一般配置过程；

*提高层次：通过参考材料，实现一个网络服务，并进行配置和测试。

# 5. 相关知识

Linux产生和发展就是与计算机网络息息相关的（比如Linux起初就是通过BBS（电子公告牌，一种网络服务）发布的，而Linux的开发也是通过计算机网络来协调进行的）。因此，Linux的网络功能非常强大，甚至很多人使用Linux的原因就是它的网络功能，Linux用作网络服务器远比它用作用户桌面系统多得多。比如Google的服务器就是使用Linux做操作系统的，Yahoo也使用Linux作为邮件服务器。学习Linux的网络应用，也就是学习如何使用Linux的网络功能，提供网络服务、保证网络安全等。

按照网络应用的对象和目的，可以把网络应用划分为网络配置、网络服务、网络安全、网络管理等。其中，网络配置解决Linux对网络硬件的支持和设置；网络服务是指通过计算机网络提供各种各样服务的应用程序；网络安全是Linux提供的针对网络安全维护、检测、防护、测试等功能；网络管理是Linux提供的对于计算机网络设备通过标准网络协议进行集中在线管理的服务。本次实验将学习配置网络地址的方法，以及学习配置、配置FTP网络服务的方法；另外，作为提高层次的要求，还提供了参考材料，以学习配置TCPWrapper网络安全服务的方法。

与Windows网络应用不同，Linux的网络应用可以不依赖于图形化的配置环境，工作在字符界面下，对一个网络管理人员来讲，字符环境的网络配置是基本功。从服务性能的角度来看，除去图形环境的系统开销，可以提高服务的性能，因此多数的Linux服务器是不安装和启动图形环境的。本实验的配置过程也在字符环境下进行，当然，Linux下也有一些图形方式的管理配置工具。

FTP服务是应用较广的网络服务，主要用于文件传输。与其他客户-服务器程序不同的是：FTP需要两个网络连接，一个用于数据传送（数据连接），一般使用TCP 21端口；一个用于控制信息传送（命令和响应，控制连接），一般使用TCP 20端口。FTP通过用户名和口令进行用户验证。下图8-1显示了FTP协议。

![FTP协议](images/8-1.png)
<center>图8-1 FTP协议</center>

TCPWrapper是Linux下的网络访问控制机制，通过对网络服务请求的监视和过滤，可以阻止指定的计算机与网络服务的通信。通常情况下，Linux对网络服务的处理流程为：Linux启动一个称为inetd（已经被xinetd[27] 所替代）的守护进程，对进入的指定网络连接进行响应。当连接建立后（通过TCP的三次握手过程），inetd启动相应的服务器程序，它自己则重新转入“睡眠”状态，等待新的连接。使用TCPWrapper后，当连接建立后，inetd程序并不直接启动服务器程序，而启动了 TCPWrapper软件（tcpd）。TCPWrapper软件对客户主机名和地址进行日志处理和其他一些检查。当检查通过后，TCPWrapper软件再启动服务器程序(in.ftpd)。下图8-2显示了inetd控制下的FTP服务流程。

![inetd控制下的FTP服务过程](images/8-2.png)
<center>图8-2 inetd控制下的FTP服务过程</center>

值得注意的是：TCPWrapper软件并不和客户、服务器程序进行交互；另外，TCPWrapper软件启动的时间是连接建立后（即TCP的三次握手过程后）。TCPWrapper提供的安全防护包括[27]：

* Access control(访问控制)

* Pevent denial of sercie attack（防止拒绝服务攻击）

* Extentive logging abilities（扩展的日志功能）

* Offload serices to a remote host（通过转向服务请求来减轻负载的服务）

* IPv6 support （支持IPv6）

* User interactions（用户交互）

本次实验只考虑访问控制功能，其他的请参考文献[27]，以及inetd或xinetd的man帮助文档。

# 6. 实验设备

实验者需要使用各自PC机上Windows操作系统的putty程序通过网络连接到Linux服务器（实验室内网为：10.11.8.17（暂时不可用），外网为：vlab.cs.swust.edu.cn），启动交互式实验指导书（浏览器方式，访问地址在启动程序中显示），学习实验相关知识操作，完成实验指导书中的“实验任务”部分。

** 实验中的命令需要以root身份运行，请使用sudo命令，命令行形式为：**

`sudo 命令`

例如，以root身份运行useradd命令，用来增加一个帐号user1，其命令行为：

`sudo useradd user1`

# 7. 实验指导

## 7.1、网卡IP地址的配置

### (1) ifconfig

在Linux下查看网卡设置的命令为ifconfig，ifconfig命令的结果为：

In [1]:
ifconfig

bash: ifconfig: command not found


这个例子中，网卡的IP地址已经事先设置过了（Linux容器的网络机制）。上述结果中，eth0为第一块物理网卡，相应的，其他物理网卡为eth1，eth2等等；lo为本地回环设备，它是一个软件网络设备，与这个设备的通信不会传输到物理网络上。从输出结果可以看到，eth0卡已经配置了IP地址了，为172.17.0.2，掩码为255.255.0.0，广播地址为172.17.0.2。

ifconfig可以手动设置网卡的网络地址，命令为：

`
ifconfig eth0 192.168.10.170 netmask 255.255.255.0
`

** 注意：因为交互式实验指导书的实验环境限制，不能在实验环境中设置网卡地址，上述命令执行后回得出如下错信息：**

SIOCSIFADDR: Operation not permitted                                                         
SIOCSIFFLAGS: Operation not permitted 

** 可以在VMware虚拟机或物理主机的Linux环境中执行上述操作。**

** 本文以下以"\*"开始的内容和相关命令因为这个原因，也无法在交互式实验指导书的实验环境中进行验证。**

### 实验任务1

请写出命令查看网络IP地址，并解释命令的输出结果。

### (2) route

另外一个使用很多的网络命令是route，用于查看、设置网络路由，-n参数表示不对IP地址进行域名解析，即直接显示为IP地址，输出结果为：

In [2]:
route -n

Kernel IP routing table
Destination     Gateway         Genmask         Flags Metric Ref    Use Iface
0.0.0.0         172.17.42.1     0.0.0.0         UG    0      0        0 eth0
172.17.0.0      0.0.0.0         255.255.0.0     U     0      0        0 eth0


\* route可以用来设置路由表，比如删除上例中的默认路由172.17.42.1，修改为默认路由192.168.10.2，具体命令为：

`
route del default 
`

`
route add default gw 192.168.10.2
`

`
route -n
`

执行结果为：

`Kernel IP routing table`

`Destination     Gateway         Genmask         Flags Metric Ref    Use Iface`

`192.168.10.0    0.0.0.0         255.255.255.0   U     0      0        0 eth0`

`0.0.0.0         192.168.10.2    0.0.0.0         UG    0      0        0 eth0`

\* 如果需要使用Linux上网，必须设置IP地址和默认路由。但是使用ifconfig和route命令设置的方式属于手工方式，系统启动后设置并不会保留。要达到保留设置的目的，需要修改配置文件，这样系统在启动后，会自动读入这些配置文件，从而达到保存配置的目的。

\* 在Ubuntu/Debian中配置IP地址和默认路由的文件是/etc/network/interfaces，文件内容为：

auto lo eth0

iface lo inet loopback

iface eth0 inet static

	address 192.168.10.170
    
	netmask 255.255.255.0
    
	gateway 192.168.10.33

第一行表示系统在启动时，需要自动启动lo和eth0两个网络设备；接下来，设置lo设备为本地回环网络设备；iface eth0 inet static一行设置eth0设备通过固定方式设置IP地址，而不是通过DHCP方式动态分配；紧接着的几行设置了IP地址（address）、网络掩码（netmask）、网关地址（gateway）。如果修改IP地址为192.168.10.171，网关地址为192.168.10.2，则/etc/network/interfaces文件为：

auto lo eth0

iface lo inet loopback

iface eth0 inet static

	address 192.168.10.171
    
	netmask 255.255.255.0
    
	gateway 192.168.10.2

\* 为了使系统读入配置文件，需要手工执行配置脚本：

#/etc/init.d/network-manager restart

\* ifconfig和route命令的使用请参考man帮助文档，/etc/network/interfaces文件的配置请参考man interfaces帮助文档，以及/usr/share/doc/ifupdown/examples下的示范文件，这些.gz的文本文件可以用zless命令打开。

### 实验任务2：

请写出命令查看网络路由，并解释命令的输出结果。

### (3) netstat

netstat命令用于显示各种网络相关信息，如网络连接，路由表，接口状态 (Interface Statistics)，其中：

`-a  选项查看所有listening状态和非listening状态的端口状态`

`-n  选项表示显示IP地址而不是主机名`

`-l  仅列出有在 Listen (监听) 状态的端口`

`-t  选项查看所有TCP端口。`

例如：

In [7]:
netstat -lnt

Active Internet connections (only servers)
Proto Recv-Q Send-Q Local Address           Foreign Address         State      
tcp        0      0 127.0.0.1:41154         0.0.0.0:*               LISTEN     
tcp        0      0 127.0.0.1:50595         0.0.0.0:*               LISTEN     
tcp        0      0 127.0.0.1:44804         0.0.0.0:*               LISTEN     
tcp        0      0 127.0.0.1:33701         0.0.0.0:*               LISTEN     
tcp        0      0 127.0.0.1:48647         0.0.0.0:*               LISTEN     
tcp        0      0 127.0.0.1:32854         0.0.0.0:*               LISTEN     
tcp        0      0 127.0.0.1:46871         0.0.0.0:*               LISTEN     
tcp        0      0 0.0.0.0:8888            0.0.0.0:*               LISTEN     
tcp6       0      0 :::8888                 :::*                    LISTEN     


上述命令的结果中，0.0.0.0:8888端口正是内网访问本实验的验指导书web服务的端口。

## 7.2、FTP服务的配置

Ubuntu/Debian提供的FTP服务器软件包括vsFTPD、ProFTPD、Pure-FTPd、netkit FTPD等，其中，vsFTPD全称为Very Secure FTP Daemon，目前被证明是很稳定的一个FTP服务器软件，比如：红帽子公司拿vsFTPD架设的服务器能提供15000人同时使用，详细介绍见其主页：http://vsftpd.beasts.org/ ；ProFTPD是一个GPL版权的可配置程度高的FTP服务器软件，使用者包括sourceforge在内的多个大型网站，详细介绍见主页：http://www.proftpd.org ； Pure-FTPd是一个高安全性的FTP服务器软件，支持多种认证方式，应用在多个高性能服务器系统中，见主页：https://www.pureftpd.org ；netkit FTPD是用于模拟网络实验的ftp服务器软件，一般不用于实际应用中。

因为交互式实验指导书的实验环境限制，本次实验选择netkit FTPD作为ftp服务器软件。

### (1) 安装和启动

在Ubuntu/Debian中，netkit FTPD服务器软件的包名称为ftpd，使用apt-get命令进行安装。

In [1]:
sudo apt-get install -y --force-yes ftpd




The following extra packages will be installed:
  libfile-copy-recursive-perl libwrap0 openbsd-inetd tcpd update-inetd
The following NEW packages will be installed:
  ftpd libfile-copy-recursive-perl libwrap0 openbsd-inetd tcpd update-inetd
0 upgraded, 6 newly installed, 0 to remove and 1 not upgraded.
Need to get 189 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu/ trusty/main libwrap0 amd64 7.6.q-25 [46.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu/ trusty/main libfile-copy-recursive-perl all 0.38-1 [20.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu/ trusty/main tcpd amd64 7.6.q-25 [23.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu/ trusty/main update-inetd all 4.43 [19.2 kB]
Get:5 http://archive.ubuntu.com/ubuntu/ trusty/main openbsd-inetd amd64 0.20091229-2ubuntu3 [30.8 kB]
Get:6 http://archive.ubuntu.com/ubuntu/ trusty/universe ftpd amd64 0.17-34 [48.9 kB]
Fetched 189 kB in 9s (19.1 kB/s)
debconf: unable to i

上述执行结果中还安装了ftpd服务器相关的软件，其中，libwrap0和tcpd包提供TCP Wrapper服务， openbsd-inetd包提供inetd服务，这几个包的关系请参考图3-2及相关描述。

启动ftpd服务有两种方法：一种方法是使用service openbsd-inetd start命令，另外一种方法是直接调用openbsd-inetd的启动脚本，/etc/init.d/openbsd-inetd restart。

举例：

In [9]:
sudo service openbsd-inetd start

 * Starting internet superserver inetd
   ...done.


上述命令结果展示启动服务成功。后面服务配置后需要重新启动服务，可以使用

`sudo service openbsd-inetd restart`

命令。

### (2) 配置ftpd服务

由于ftpd服务是由inetd服务来启动的，所以需要配置inetd的启动配置文件/etc/inetd.conf，以启动ftpd服务。

In [8]:
cat /etc/inetd.conf

# /etc/inetd.conf:  see inetd(8) for further informations.
#
# Internet superserver configuration database
#
#
# Lines starting with "#:LABEL:" or "#<off>#" should not
# be changed unless you know what you are doing!
#
# If you want to disable an entry so it isn't touched during
# package updates just comment it out with a single '#' character.
#
# Packages should modify this file by using update-inetd(8)
#
# <service_name> <sock_type> <proto> <flags> <user> <server_path> <args>
#
#:INTERNAL: Internal services
#discard		stream	tcp	nowait	root	internal
#discard		dgram	udp	wait	root	internal
#daytime		stream	tcp	nowait	root	internal
#time		stream	tcp	nowait	root	internal

#:STANDARD: These are standard services.
ftp		stream	tcp	nowait	root	/usr/sbin/tcpd	/usr/sbin/in.ftpd

#:BSD: Shell, login, exec and talk are BSD protocols.

#:MAIL: Mail, news and uucp services.

#:INFO: Info services

#:BOOT: TFTP service is provided primarily for booting.  Most sites
#       run this only on machines

上面的配置文件中，启动ftpd服务的一行是：

ftp		stream	tcp	nowait	root	/usr/sbin/tcpd	/usr/sbin/in.ftpd

表示ftp服务由/usr/sbin/tcpd软件（TCPWrapper软件）启动，参数为/usr/sbin/in.ftpd。这也证明了前述inetd和ftpd的关系。（见图8-2及相应的叙述）

与ftpd配置相关的文件包括：

* /etc/ftpusers

不允许使用ftp服务的用户列表；

* /etc/ftpchroot

需要被限制访问路径（称为chroot，即仅访问用户主目录）的用户列表；

* /etc/ftpwelcome

连接FTP服务后显示的欢迎信息；

* /etc/motd

用户登录后看见的欢迎信息；

* /etc/nologin

如果该文件存在，则FTP服务不可使用，仅显示该文件中的信息并退出。

ftpd认证登录用户身份的步骤包括：
 
1、登录用户必须在/etc/passwd中；

后面的步骤中，需要先使用adduser命令创建一个用户。
 
2、登录用户不能出现在文件/etc/ftpusers中；
 
3、登录用户的shell必须是/etc/shells文件中的一个；
 
4、如果登录用户出现在/etc/ftpchroot中，则登录FTP服务后，只能访问该用户的主目录；
 
5、如果需要允许匿名登录（ftp或anonymous用户），则ftp帐号需要出现在/etc/passwd文件中，并设置密码，匿名登录ftp服务时，密码可以是任意的。

详细信息请参考ftpd的帮助命令： man ftpd

### (3) 验证

验证ftp服务有两种方法，一种是使用netstat命令查看TCP 21端口是否处于监听（LISTEN）状态；一种是使用ftp客户端软件看是否可以访问文件，如文件管理器、Filezilla、ftp命令等。

Ubuntu/Debian下安装ftp命令的方法是执行apt-get install ftp命令，Windows系统命令行下默认已经安装了ftp命令。

## 7.3、TCPWrapper的配置

TCPWrapper的配置文件为/etc/hosts.allow，/etc/hosts.deny，这两个文件的格式都一样，只是作用不同：hosts.allow是允许访问的控制文件，hosts.deny是禁止访问的控制文件，hosts.allow的内容先被读取，然后是hosts.deny，默认为允许。

hosts.allow文件中，空行和#开头的内容为注释内容，它的格式为：

daemon_list : client_list [ : shell_command ]

daemon_list是一个或多个守护进程（也就是服务程序）的名称，也可以是通配符，列表由空格或符号“,”分隔。

client_list是匹配客户主机名称或地址的一个或多个主机名、主机地址、模式或通配符，列表由空格或符号“,”分隔。

形如daemon@host用来扩展daemon_list中服务器端模式，形如user@host用来扩展client_list中客户端的用户名。

shell_command是满足条件需要执行的命令。

上述配置行中可以包括一些特殊意义的符号：

ALL 匹配所有。

LOCAL 匹配不含“.”字符的主机名。

UNKNOWN 匹配不知道名字的用户和不知道名字的主机。这个模式需要谨慎使用，因为主机名的解析通常跟名字服务程序是否正常运行有关。当软件无法辨认出网络类型时，网络地址就是“未认识”。

KNOWN 与UNKNOWN相反。

PARANOID 匹配主机名与地址不匹配的主机。如果tcpd使用-DPARANOID参数编译(默认方式)，tcpd会在读控制表之前丢掉服务请求。

举例：

(1) 关闭大多数的服务

任务：关闭对所有主机的服务，除了/etc/hosts.allow中允许的之外。

/etc/hosts.deny:

ALL: ALL


/etc/hosts.allow:

ALL: LOCAL @some_netgroup

ALL: .foobar.edu EXCEPT terminalserver.foobar.edu

第一个规则允许从本地域（主机名中无符号“.”）以及some_netgroup网络组成员的访问。第二个规则允许foobar.edu域中（注意域名前面的符号“.”）除了主机 terminalserver.foo-bar.edu的所有主机的访问。

(2) 开放大多数的配置

任务：除了hosts.deny中拒绝的服务，其他的都开放。

/etc/hosts.deny:

ALL: some.host.name, .some.domain

ALL EXCEPT in.ftpd: other.host.name, .other.domain

第一个规则禁止some.host.name主机和some.domain域内的所有主机访问所有服务.第二个规则仍然禁止other.host.name主机和.other.domain域内的所有主机访问除in.ftpd服务。

### 实验任务3：

按照实验步骤，在指导书实验环境上配置使用FTPD服务，写出详细配置过程和结果。FTP服务的要求为：

1) 设置欢迎信息为"Welcome to our server!"

2) 一般用户（使用adduser创建的用户）仅能访问用户主目录；

3) 任意指定一台计算机（比如你同桌的计算机），使其无法访问FTP服务。

# 8 实验任务

完成指导书中的3个任务，将过程的结果反映在实验报告中。

# 9 实验思考

1、当hosts.all和hosts.deny中的配置有矛盾的时候，网络访问控制如何进行？